# Geodesic Runtime Comparison

In [115]:
import pickle

from tabulate import tabulate

In [127]:
path = '../timing/'
manifold = "Sphere"

## Printing Table Values

In [128]:
save_path = ''.join((path, manifold, '.pkl'))
with open(save_path, 'rb') as f:
    estimates = pickle.load(f)

In [129]:
dims = list(estimates.keys())

mu_time = [estimates[key3][key2]['mu_time'] for key3,v3 in estimates.items() for key2,v2 in v3.items()]
std_time = [estimates[key3][key2]['std_time'] for key3,v3 in estimates.items() for key2,v2 in v3.items()]
length = [estimates[key3][key2]['length'] for key3,v3 in estimates.items()for key2,v2 in v3.items()]
iter_num = [estimates[key3][key2]['iterations'] for key3,v3 in estimates.items() for key2,v2 in v3.items()]
grad_norm = [estimates[key3][key2]['grad_norm'] for key3,v3 in estimates.items() for key2,v2 in v3.items()]
max_iter = [estimates[key3][key2]['max_iter'] for key3,v3 in estimates.items() for key2,v2 in v3.items()]
tol = [estimates[key3][key2]['tol'] for key3,v3 in estimates.items() for key2,v2 in v3.items()]
method = [key2 for key3,v3 in estimates.items() for key2,v2 in v3.items()]

N_methods = len(list(set(method)))
dims = sum([[d]*N_methods for d in dims], [])

TypeError: JAX does not support string indexing; got idx=('mu_time',)

In [130]:
names = [r"Dim", r"Method", 
        r"Length", r"Iterations", r"grad_norm", r"mean time",
        r"std time", r"max_iter", r"tol"]

table_dict = {}
table_dict['Dim'] = dims
table_dict['Method'] = method
table_dict['Length'] = length
table_dict['Iterations'] = iter_num
table_dict['grad_norm'] = grad_norm
table_dict['mu_time'] = mu_time
table_dict['std_time'] = std_time
table_dict['max_iter'] = max_iter
table_dict['tol'] = tol

print(tabulate(table_dict, headers=names, floatfmt=".4f"))


Dim               Method              Length    Iterations    grad_norm    mean time    std time    max_iter     tol
----------------  ----------------  --------  ------------  -----------  -----------  ----------  ----------  ------
gradient          gradient            3.4849     1000.0000       0.0466      29.9518      0.4224        1000  0.0001
init              init                3.4841                                                            1000  0.0001
ground_truth      ground_truth                                                                          1000  0.0001
geodesic_control  geodesic_control    2.9080       10.0000       0.0000       0.3760      0.0061        1000  0.0001
adam              adam                2.9176     1000.0000       0.2378       5.3114      0.1841        1000  0.0001
sgd               sgd                 2.9176     1000.0000       0.2378       5.2795      0.1493        1000  0.0001
                  BFGS                2.9080      194.0000      